In [4]:
import numpy as np
from Ant_quant import *
env = gym.make('Ant-v5')
cfg = {
        "quant_act": True,
        "quant_weights": True,
        "activation": "relu",
    }
model = PPO(
        policy=QuantizedActorCriticPolicy,
        env=env,
        policy_kwargs={"cfg": cfg},
        verbose=1,
        tensorboard_log="./ppo_tensorboard_ant/"
    )

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


# Load the model

In [31]:
model_path = "/home/ritwik/MuJoCo_Quant/logs_ant/best_model_fixscale_1/best_model.zip"
import torch as th
import zipfile
import io
from Ant_quant import *
# Open the zip and load policy.pth
with zipfile.ZipFile(model_path, "r") as archive:
    with archive.open("policy.pth", "r") as f:
        state_dict = th.load(io.BytesIO(f.read()), map_location="cpu")

state_dict.keys()

odict_keys(['log_std', 'mlp_extractor.policy_net.act1.x_min', 'mlp_extractor.policy_net.act1.x_max', 'mlp_extractor.policy_net.act1.act_scaling_factor', 'mlp_extractor.policy_net.act1.pre_weight_scaling_factor', 'mlp_extractor.policy_net.act1.identity_weight_scaling_factor', 'mlp_extractor.policy_net.fc1.weight', 'mlp_extractor.policy_net.fc1.bias', 'mlp_extractor.policy_net.fc1.fc_scaling_factor', 'mlp_extractor.policy_net.fc1.weight_integer', 'mlp_extractor.policy_net.fc1.bias_integer', 'mlp_extractor.policy_net.act2.x_min', 'mlp_extractor.policy_net.act2.x_max', 'mlp_extractor.policy_net.act2.act_scaling_factor', 'mlp_extractor.policy_net.act2.pre_weight_scaling_factor', 'mlp_extractor.policy_net.act2.identity_weight_scaling_factor', 'mlp_extractor.policy_net.fc2.weight', 'mlp_extractor.policy_net.fc2.bias', 'mlp_extractor.policy_net.fc2.fc_scaling_factor', 'mlp_extractor.policy_net.fc2.weight_integer', 'mlp_extractor.policy_net.fc2.bias_integer', 'mlp_extractor.policy_net.act3.x_mi

# Get the scales and weights

In [32]:
def batch_frexp_refactored(inputs):
    """
    Decompose the scaling factor into mantissa and twos exponent, ensuring:
    - Mantissa lies in [0, 2^16)
    - Exponent remains meaningful (adjusted to match the scaled mantissa)

    Parameters:
    ----------
    inputs: Tensor
        Scaling factor

    Returns:
    -------
    mantissa: Tensor
    exponent: Tensor
    """
    shape_of_input = inputs.size()

    # Flatten the input tensor to a 1D tensor
    inputs = inputs.view(-1)

    # Decompose into mantissa and exponent
    output_m, output_e = np.frexp(inputs.cpu().numpy())

    # Prepare adjusted mantissa and exponent lists
    tmp_m = []
    tmp_e = []

    # Define the bias to scale mantissa to [0, 2^16)
    scale_factor = 16  # 2^16

    for idx, (m, e) in enumerate(zip(output_m, output_e)):
        # Scale mantissa from [0.5, 1) to [0, 2^16)
        int_m_shifted = int(Decimal(m * (2 ** scale_factor)).quantize(Decimal('1'), rounding=decimal.ROUND_HALF_UP))

        # Adjust the exponent to account for the scaling of the mantissa
        e -= scale_factor
        e= e*-1
        e = max(0, min(255, e))
        # Ensure mantissa and exponent are within valid bounds
        if int_m_shifted >= 2 ** 16:  # Cap mantissa at maximum (65535)
            int_m_shifted = 2 ** 16 - 1

        elif inputs[idx] >= 65535:
            int_m_shifted = 65535
            e = 0

        tmp_m.append(int_m_shifted)
        tmp_e.append(e)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # tensor = torch.tensor(0.).to(device)

    return torch.from_numpy(np.array(tmp_m)).to(device).view(shape_of_input), \
           torch.from_numpy(np.array(tmp_e)).to(device).view(shape_of_input)


In [33]:
Sx2

tensor([0.0037])

In [34]:
Sx = state_dict['mlp_extractor.policy_net.act1.act_scaling_factor']
Sw = state_dict['mlp_extractor.policy_net.fc1.fc_scaling_factor']
Sx1 = state_dict['mlp_extractor.policy_net.act2.act_scaling_factor']
Sw1 = state_dict['mlp_extractor.policy_net.fc2.fc_scaling_factor']
# Sx2 = state_dict['mlp_extractor.policy_net.act3.act_scaling_factor']
Sw2 = state_dict['action_net.fc1.fc_scaling_factor']
Sx2 = state_dict['action_net.act1.act_scaling_factor']
Sw3= state_dict['action_net.fc1.fc_scaling_factor'] 
Sx3 = state_dict['action_net.act2.act_scaling_factor']

In [35]:
Sx2*Sw3

tensor([2.1747e-06])

In [36]:
import numpy as np
m0, e0 = batch_frexp_refactored(Sx)
m1, e1 = batch_frexp_refactored(Sw*Sx/Sx1)
m2, e2 = batch_frexp_refactored(Sw1*Sx1/Sx2)
m3, e3 = batch_frexp_refactored(Sx2*Sw2/Sx3)

w1 = state_dict['mlp_extractor.policy_net.fc1.weight_integer']
b1 = state_dict['mlp_extractor.policy_net.fc1.bias_integer']
w2 = state_dict['mlp_extractor.policy_net.fc2.weight_integer']
b2 = state_dict['mlp_extractor.policy_net.fc2.bias_integer']
w3 = state_dict['action_net.fc1.weight_integer']
b3 = state_dict['action_net.fc1.bias_integer']

In [24]:
env = gym.make('Ant-v5')
obs = env.reset()

In [30]:
obs

(array([ 0.72609608,  0.97502791, -0.00550536,  0.01237795,  0.07291184,
        -0.0789343 , -0.0252412 ,  0.09569585,  0.08525837,  0.07520057,
         0.04386474,  0.00606424, -0.02980362,  0.0626841 , -0.07022213,
        -0.03829926, -0.08660343, -0.12020378, -0.04911877, -0.10811243,
        -0.15067203, -0.04501276,  0.05172145,  0.01603967, -0.02342684,
         0.00700051,  0.03747318,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0. 

In [37]:
# Forward pass
obs_=env.reset()
# import torch
# obs_quant = torch.round(torch.tensor(obs_req)*2**e0/m0).to(torch.float32)
# print(obs_quant)
# a1 = torch.round((torch.matmul(obs_quant,w1.T)+b1)*m1/2**e1)
# a1 = torch.relu(a1)
# a1 = torch.clip(a1, 0, 255)
# print(a1)
# a2 = torch.round((torch.matmul(a1,w2.T)+b2)*m2/2**e2)
# a2 = torch.relu(a2)
# a2 = torch.clip(a2, 0, 255)
# print(a2)
# a3 = torch.round((torch.matmul(a2,w3.T)+b3)*m3/2**e3)
# a3 = torch.clip(a3, -127, 127)
# print(a3)

def forward_pass(obs):
    obs_quant = torch.round(torch.tensor(obs)*2**e0/m0).to(torch.float32)
    a1 = torch.round((torch.matmul(obs_quant,w1.T)+b1)*m1/2**e1)
    a1 = torch.relu(a1)
    
    a2 = torch.round((torch.matmul(a1,w2.T)+b2)*m2/2**e2)
    a2 = torch.relu(a2)
    a3 = torch.round((torch.matmul(a2,w3.T)+b3)*m3/2**e3)

    return a3




In [38]:
obs[0]

0.3723589072438309

In [39]:
forward_pass(obs)*2**e3/m3*Sw3*Sx2

tensor([-0.0000,  0.0249, -0.0373, -0.0000, -0.0373,  0.0622,  0.0000,  0.0498])

In [21]:
forward_pass(obs)*Sx3

tensor([-0.0000,  0.0124, -0.0498, -0.0124, -0.0373,  0.0498,  0.0124,  0.0498])

In [28]:
obs

array([ 7.71079313e-01,  9.93715344e-01,  9.75758965e-03, -7.45706249e-02,
        8.29085402e-02, -1.21863137e-01,  8.64816961e-01, -8.36181849e-03,
       -8.32872007e-01, -6.47045660e-02, -8.74590321e-01,  1.59677392e-02,
        8.58528053e-01, -1.55063724e-01, -4.02453615e-02, -5.48605858e-01,
        4.65907036e-02, -5.52429561e-03,  1.04598326e-01, -6.70952645e-01,
        8.53270053e+00, -4.74178474e-01, -7.75348259e+00,  2.63192802e-01,
       -8.78298936e+00,  4.54328995e-02,  8.31510100e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [40]:
import imageio
timestep=0
obs = env.reset()
total_reward= 0
done = False
while not done and timestep < 1000:
    # print("YY")
    if timestep==0:
        action = forward_pass(obs[0])*Sx3
    else:
        action = forward_pass(obs)*Sx3
    action = action.cpu().numpy()
    obs, reward, terminated, truncated, info = env.step(action)
    obs_req = obs
    # break
    done = terminated or truncated
    total_reward += reward
    timestep+=1
print(total_reward)

1246.2433450550734


In [156]:
a2

tensor([-2155379.,   758985.,   464912., -1899421.,  1499139., -1574465.,
        -4345860., -3861966., -7840385.,  4891974.,   700833.,  1490702.,
        -5453304.,  4757290.,   450450., -7211564., -4629086., -1174935.,
        -3139589., -1614839., -3562318.,  4347969., -2386027.,  2227541.,
         4704412., -2553552., -3125277., -5519289.,   197204.,  1077765.,
         2324260.,  2808908., -1557140.,  1190904., -9619285.,   516887.,
         2385877.,  -317876.,  5409765.,  4031449., -2605678.,  4518206.,
         -890655.,  6015688., -1880891.,  6667710., -4437004., -4257427.,
        -1399558.,  3944674.,  4794351.,  5548214., -3212806., -1981376.,
         -470486.,  2264300., -3334683.,  -871824.,  2095420.,  6509073.,
         3296116.,  3142903., -3046185.,  1013888., -2172553., 10089771.,
         -605169., -5495636., -8616695.,  -987826., -4678500., -4386234.,
        -3623032., -2268218.,  -952874.,  -634396., -4670666.,  9668699.,
        -3375209.,   668443.,  -294374

In [39]:
obs_quant_save = obs_quant.cpu().numpy().reshape(-1, 1)
a1_save = a1.cpu().numpy().reshape(-1, 1)
a2_save = a2.cpu().numpy().reshape(-1, 1)
a3_save = a3.cpu().numpy().reshape(-1,1)

# 

In [ ]:


w1_save = w1.cpu().numpy()
w2_save = w2.cpu().numpy()
w3_save = w3.cpu().numpy()

b1_save = b1.cpu().numpy().reshape(-1,1)
b2_save = b2.cpu().numpy().reshape(-1,1)
b3_save = b3.cpu().numpy().reshape(-1,1)

In [113]:
obs_quant_save.shape

(105, 1)

In [122]:
m0.reshape(-1,1)

tensor([[39279]])

In [40]:
os.mkdir("Activation")
path1 = "./Activation/"
np.savetxt(path1+"obs_quant.csv", obs_quant_save.T, delimiter=",", fmt="%.6f")
np.savetxt(path1+"a1.csv", a1_save.T, delimiter=",", fmt="%.6f")
np.savetxt(path1+"a2.csv", a2_save.T, delimiter=",", fmt="%.6f")
np.savetxt(path1+"a3.csv", a3_save.T, delimiter=",",fmt="%.6f")

In [170]:
import os
# os.mkdir("Activation")

os.mkdir("Parameters")
path1 = "./Activation/"
path2 = "./Parameters/"
np.savetxt(path1+"obs_quant.csv", obs_quant_save.T, delimiter=",", fmt="%.6f")
np.savetxt(path1+"a1.csv", a1_save.T, delimiter=",", fmt="%.6f")
np.savetxt(path1+"a2.csv", a2_save.T, delimiter=",", fmt="%.6f")
np.savetxt(path1+"a3.csv", a3_save.T, delimiter=",",fmt="%.6f")
np.savetxt(path2+"fc1_weight.csv", w1_save.T, delimiter=",",fmt="%.6f")
np.savetxt(path2+"fc2_weight.csv", w2_save.T, delimiter=",",fmt="%.6f")
np.savetxt(path2+"fc3_weight.csv", w3_save.T, delimiter=",",fmt="%.6f")
np.savetxt(path2+"fc1_bias.csv", b1_save.T, delimiter=",",fmt="%.6f")
np.savetxt(path2+"fc2_bias.csv", b2_save.T, delimiter=",",fmt="%.6f")
np.savetxt(path2+"fc3_bias.csv", b3_save.T, delimiter=",",fmt="%.6f")
# np.save


In [127]:
import os
os.mkdir("Scales")
path = "./Scales/"

np.savetxt(path+"m0.csv", m0.reshape(-1,1).cpu().numpy(), delimiter=",",fmt="%.6f")
np.savetxt(path+"e0.csv", e0.reshape(-1,1).cpu().numpy(), delimiter=",",fmt="%.6f")
np.savetxt(path+"m1.csv", m1.reshape(-1,1).cpu().numpy(), delimiter=",",fmt="%.6f")
np.savetxt(path+"e1.csv", e1.reshape(-1,1).cpu().numpy(), delimiter=",",fmt="%.6f")
np.savetxt(path+"m2.csv", m2.reshape(-1,1).cpu().numpy(), delimiter=",",fmt="%.6f")
np.savetxt(path+"e2.csv", e2.reshape(-1,1).cpu().numpy(), delimiter=",",fmt="%.6f")
np.savetxt(path+"m3.csv", m3.reshape(-1,1).cpu().numpy(), delimiter=",",fmt="%.6f")
np.savetxt(path+"e3.csv", e3.reshape(-1,1).cpu().numpy(), delimiter=",",fmt="%.6f")

In [5]:
import torch
params = torch.load("/home/ritwik/MuJoCo_Quant/logs_ant/best_model_fixscale_1/best_model.zip", map_location="cpu")

RuntimeError: [enforce fail at inline_container.cc:176] . file in archive is not in a subdirectory: data

In [3]:
model.value_net


NameError: name 'model' is not defined